In [1]:
import pandas as pd
from datetime import datetime
import time 
import requests
from datetime import timedelta, datetime
from functions import *

In [2]:
date_today = datetime.now()
date_today =date_today.strftime("%Y-%m-%d")

In [4]:
cities = ['Amsterdam']


data_air_quality = [get_air_quality_data(city) for city in cities]

data_weather = [get_weather_data(city, date_today) for city in cities]

In [5]:
data_air_quality

[['2023-01-14', 15, 6, 30.9, 1.2, 0.1, 1.7, 31]]

In [6]:
data_weather

[['2023-01-14',
  11.0,
  6.0,
  8.1,
  11.0,
  1.4,
  5.0,
  6.4,
  88.8,
  7.5,
  100.0,
  33.33,
  79.6,
  47.3,
  215.3,
  1001.9,
  75.5,
  8.9,
  4.3,
  0.3,
  0.0,
  'Rain, Partially cloudy']]

In [37]:
df_air_quality = get_air_quality_df(data_air_quality)

df_air_quality

,date,pm25,pm10,o3,no2,so2,co,aqi
0,1673564400000,28,10,26.6,2.6,0.2,1.8,28


In [38]:
aqi=[]
for i in range(0,len(df_air_quality)):
    row=df_air_quality.iloc[i,1:-2]
    aqi.append(max(row))
df_air_quality['aqi']=aqi

In [48]:
df_air_quality

,date,pm25,pm10,o3,no2,so2,co,aqi
0,1673564400000,28,10,26.6,2.6,0.2,1.8,28


In [46]:
df_air_quality['aqi'] = df_air_quality['aqi'].astype(int)

In [47]:
df_air_quality.dtypes

date      int64
pm25      int64
pm10      int64
o3      float64
no2     float64
so2     float64
co      float64
aqi       int64
dtype: object

In [41]:
df_weather = get_weather_df(data_weather)

df_weather.head()

,date,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,...,windgust,windspeed,winddir,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,conditions
0,1673564400000,9.8,7.7,8.9,7.7,3.3,4.9,4.8,75.3,1.1,...,74.2,45.9,245.9,1004.0,71.3,11.2,11.9,1.0,1.0,"Rain, Partially cloudy"


In [43]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

air_quality_fg = fs.get_or_create_feature_group(
    name = 'air_quality_fg',
    version = 1
)
weather_fg = fs.get_or_create_feature_group(
    name = 'weather_fg',
    version = 1
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5419
Connected. Call `.close()` to terminate connection gracefully.


In [11]:
air_quality_fg.insert(df_air_quality)

Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/5419/jobs/named/air_quality_fg_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7fad9cf5caf0>, None)

In [12]:
weather_fg.insert(df_weather)

Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/5419/jobs/named/weather_fg_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7fad9d15e3a0>, None)